# SAR Dataset Accessibility

### Work in progress notebook

There are several ways to find and access the SAR datasets. These are the datasets which contain the calibrated geophysical range Doppler frequency shift retrievals from the ENVISAT ASAR wide-swath acqusitions obtained between 2002 and 2012. In the following a description of some selected methods for finding and extracting these datasets are presented.

### Find Data Through Web Search


All data is freely available and can be found in the MET Norway thredds catalog: https://thredds.met.no/thredds/catalog.html

![Thredds Dataset Overview](../images/Thredds_Dataset_Overview_image_cropped.png)

The ENVISAT ASAR datasets are located at: https://thredds.met.no/thredds/catalog/remotesensingenvisat/asar-doppler/catalog.html

Or just following this folder structure: Observations/Remotesensing_archive/ENVISAT_ASAR_Doppler:

![ENVISAT ASAR Doppler Overview](../images/ENVISAT_ASAR_Doppler_Overview_cropped.png)

Entering the subfolder, the separate netCDF-files are found under separate pathways dependent on their respective dates. Wanting to access the files for a specific date the following structure is followed: YEAR/MONTH/DAY

Underneath the path to 2012/01/27 is shown:

![ASAR 2012 overview](../images/ASAR_2012_overview_cropped.png)

![ASAR 2012/01 overview](../images/ASAR_2012_01_overview_cropped.png)

![ASAR 2012/01/27 overview](../images/ASAR_2012_01_27_overview_cropped.png)

The entire list of files from the specified date are then accessible (the list goes on).

### Access Data

Upon accessing a specific netCDF-file four different "Access"-options are available. These are "OPENDAP", "HTTPServer", "WCS" and "WMS". 

![ASAR 2012-01-27 netCDF overview.png](../images/ASAR_2012_01_27_netCDF_overview.png)

The separate options are explained in the following. In these examples the netCDF file "ASA_WSDV2PRNMI20120127_215005_000614583111_00101_51839_0000.nc" (the uppermost file under 2012/01/27) is used as an example.

#### OPENDAP - Using xarray:

The data is easily accessed through OPENDAP by the use of the xarray python package. Below is a an example on how to use xarray to open and investigate a desired dataset.
This procedure makes it easy to inspect the Dimensions, Coordinates, Data Variables, Indexes and Attributes of the dataset in question. 

In [1]:
# Import the required package: xarray
import xarray as xr

# Providing the OPENDAP-url
OPENDAP_url = 'https://thredds.met.no/thredds/dodsC/remotesensingenvisat/asar-doppler/2012/01/27/ASA_WSDV2PRNMI20120127_215005_000612433111_00101_51839_0000.nc'

# Using xarray to open the dataset using the OPENDAP-url
ds = xr.open_dataset(OPENDAP_url)

# Investigating the data as a xarray.Dataset 
ds

<xarray.Dataset> Size: 39MB
Dimensions:             (y: 602, x: 851)
Dimensions without coordinates: y, x
Data variables: (12/19)
    sensor_view_angle   (y, x) float32 2MB ...
    incidence_angle     (y, x) float32 2MB ...
    sensor_azimuth      (y, x) float32 2MB ...
    sigma0_VV           (y, x) float32 2MB ...
    topographic_height  (y, x) float32 2MB ...
    valid_land_doppler  (y, x) float32 2MB ...
    ...                  ...
    u_range             (y, x) float32 2MB ...
    std_u_range         (y, x) float32 2MB ...
    wind_direction      (y, x) float32 2MB ...
    wind_speed          (y, x) float32 2MB ...
    longitude           (y, x) float32 2MB ...
    latitude            (y, x) float32 2MB ...
Attributes: (12/75)
    GDAL:                       GDAL 3.2.1, released 2020/12/29
    acknowledgement:            GSAR IDL pre-processing software was develope...
    azimuth_start_time:         2012-01-27T21:50:05.173597
    contributor_email:          jeong-won.park@kopri.re.kr, hjoh@norceresearc...
    contributor_institution:    Korea Polar Research Institute (KOPRI), NORCE...
    contributor_name:           Jeong-Won Park, Harald Johnsen, Geir Engen
    ...                         ...
    ysamplefreq_slc:            1711.9144385026736
    ysize:                      629
    ytime:                      2012-01-27T21:50:05.173597Z
    ytime_slc:                  2012-01-27T21:50:04.077748
    summary_no:                 Kalibrert geofysisk Dopplerskift fra ENVISAT ...
    title_no:                   Kalibrert geofysisk ENVISAT ASAR Dopplerskift...

### Find Data Through CSW (Catalog Service on the Web)

Having problems finding the datasets using CSW. I am able to find the datasets used as examples here https://github.com/metno/esa-coscaw-data-search, but not the Envisat ASAR data we want to find here... (just at staging site for now I assume...) - Note for self - See csw_test.ipynb for code example!

### How to Visualize Data with WMS (Web Map Service)

#### By the Use of data.met.no

Using data.met.no it is possible to both find and visualise datasets. The web search interface can be accessed from the "Data Catalog" menu item, or directly at https://data.met.no/metsis/search (BUT I CANT FIND THE ENVISAT ASAR DATA HERE... Assuming that these are not available here as of yet (only on staging page?)). As seen below the search interface consists of a map and a series of filters.

![Data Catalog Overview](../images/DataMetNo_Data_Catalog_Overview_image.png)

The map provides a pagination of available datasets in the metadata catalog [max/min longitude/latitude rectangle], sorted to showcase the latest additions first. One can also interact with the map to better diplay the results, and to perform data search.

* "Select Projection" located just above the map can be altered to change the map projection. "Spatial filter" can be set to both "Within" and "Intersects".
* The "Create bounding box"-button enables to set a bounding box directly on the map and works as a filter on the results.
* The "Reset Search"-button clears the filters and starts a new search.
* The "Reset Map"-button resets the map.

Map widgets allows direct interaction with the map:

* +/-:                     Zoom in/out.
* E:                       Zooms to the extent of the displayed datasets.
* Menu tag:                Opens side panel where WMS Layers, Features and Base Layers can be altered.
* Magnifying glass:        Enables searching for location names.
* '>>':                    Showing the location in an overview world map.
* Upper right hand widget: Full screen mode

Search filters can also be used to find the desired datasets. The results are updated dynamically when filters are selected. These allows:

* A full text search block where the options "Contains all of these words" and "Contains any of these words" are eligible.
* Start and end date of the desired datasets.
* An option named "Has children" to determine whethere datasets are parents with children (i.e. records of the same type).
* The desired sorting mechanism.
* Isotopic categories: The general subjects for which the geospatial data may be relevant, as defined by the ISO standard.
* Keywords: Keywords from a controlled vocabulary.
* Activity type: The nature of the dataset(s) generation process (Numerical Simulation, Climate Indicator, In Situ Land-based station, Space Borne Instrument).
* Project: Datsets related to a certain project.

By clicking the "Reset"-button all filters are removed and a new search can be initiated.




#### By the Use of QGIS

As the MET Norway's S-ENDA CSW catalog service is also available through QGIS, datasets can be found and inspected as follows:

1. From the menu select "Web > MetaSearch > MetaSearch".
2. Select "Services > New".
3. For the name type "data.csw.met.no".
4. For the URl type "https://data.csw.met.no".


To get a list of the available datasets there is the option to add different search parameters under the "Search" tab. By further clicking at "Search" the available datasets which correspond to the different paramteres will be displayed. 

#### By the Use of geoweb(?)